In [1]:
import numpy as np
import pandas as pd

from konlpy.tag import Komoran

from collections import Counter

In [2]:
news_data = pd.read_csv('../Data/SBS.csv')

# news_data.loc[news_data.date.str.contains('2022.10.01')] # 4282

news_data = news_data.iloc[4282:, :]
len(news_data)

6552

In [3]:
categories = news_data.category.unique()

len(categories)

7

In [4]:
categories

array(['스포츠', '국제', '사회', '생활·문화', '경제', '정치', '연예'], dtype=object)

In [5]:
news = []

for category in categories:
    try:
        news.append(news_data.groupby('category').get_group(category).content)
    except (KeyError):
        pass

In [6]:
len(news)

7

In [7]:
komoran = Komoran()

contents_nouns = []

In [18]:
for news_category in news:
    contents_nouns_category = []
    
    for content in news_category:
        poses = komoran.pos(content)
        
        for pos in poses:
            if len(pos[0]) < 3:
                continue
            
            if pos[-1] == 'NNP':
                contents_nouns_category.append(pos[0])
    
    contents_nouns.append(contents_nouns_category)

In [19]:
len(contents_nouns)

7

In [26]:
nouns_mean_category =   [9, 9, 10, 4, 6, 12, 4]
mean_idx = 0

nouns_cnt = []

for contents_nouns_category in contents_nouns:
    cnt = Counter(contents_nouns_category)
    nouns_cnt_category = cnt.most_common(len(cnt))
    
    nouns_cnt_sep = []
    
    for noun_cnt in nouns_cnt_category:
        if noun_cnt[-1] > nouns_mean_category[mean_idx]:
            nouns_cnt_sep.append(noun_cnt)
    
    mean_idx += 1
    
    nouns_cnt.append(nouns_cnt_sep)

In [27]:
len(nouns_cnt)

7

In [28]:
categories = ['sports', 'international', 'social', 'culture', 'economy', 'politics', 'entertainments']
category_idx = 0

for nouns_cnt_category in nouns_cnt:
    df = pd.DataFrame(nouns_cnt_category, columns=['noun', 'cnt'])
    df.to_csv('../Data/SBS_nouns_cnt_{0}.csv'.format(categories[category_idx]), index=False)
    
    category_idx += 1

In [18]:
category_idx = 0

print('-mean-')
for nouns_cnt_category in nouns_cnt:
    sum = 0
    
    for noun_cnt in nouns_cnt_category:
        sum += noun_cnt[-1]
        
    print('{}:'.format(categories[category_idx]), round(sum / len(nouns_cnt_category), 0))
    
    category_idx += 1

-mean-
sports: 9.0
international: 9.0
social: 10.0
culture: 4.0
economy: 6.0
politics: 12.0
entertainments: 4.0


In [17]:
category_idx = 0

print('-median-')
for nouns_cnt_category in nouns_cnt:
    cnt = []
    
    for noun_cnt in nouns_cnt_category:
        cnt.append(noun_cnt[-1])
    
    if len(cnt) % 2 ==0:
        cnt_idx = len(cnt) // 2
        
        print('{}:'.format(categories[category_idx]), (cnt[cnt_idx -1] + cnt[cnt_idx]) / 2)
    else:
        cnt_idx = len(cnt) // 2 + 1
        
        print('{}:'.format(categories[category_idx]), cnt[cnt_idx])
    
    category_idx += 1

-median-
sports: 2
international: 2
social: 2.0
culture: 1
economy: 2.0
politics: 2
entertainments: 2.0
